# Melon 주간 Top 100 구하기

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request

In [2]:
import requests
url ='https://www.melon.com/chart/week/index.htm'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
req = requests.get(url, headers = headers)
html = req.text

In [3]:
soup = BeautifulSoup(html,'html.parser')

### 차트 기간

In [4]:
soup.select_one('.yyyymmdd').get_text()

'\r\n\t\t\t\t\t\t\t2020.12.21 ~ 2020.12.27\r\n\t\t\t\t\t\t'

In [5]:
date = soup.select_one('.yyyymmdd').get_text().strip()
date

'2020.12.21 ~ 2020.12.27'

In [6]:
start_day = date[:10]
end_day = date[13:]
start_day, end_day

('2020.12.21', '2020.12.27')

### 곡 리스트 찾기

In [7]:
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')
len(trs)

100

In [8]:
rank = int(trs[0].select_one('.rank').string)
print(rank)

1


In [9]:
for i in [0, 1, 2, 19]:
    entry = trs[i].select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100
    if len(spans) == 3:
        diff = int(spans[2].string)
        if diff != 0:
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1
    print(diff)

0
4
0
-4


In [10]:
info = trs[0].select_one('.ellipsis.rank01')
title = info.find('a').string
info = trs[0].select_one('.ellipsis.rank02')
singer = info.find('a').string
info = trs[0].select_one('.ellipsis.rank03')
album = info.find('a').string
title, singer, album

('VVS (Feat. JUSTHIS) (Prod. GroovyRoom)', '미란이 (Mirani)', '쇼미더머니 9 Episode 1')

#### DataFrame으로 만들기

In [11]:
ranks = []; diffs = []; 
titles = []; singers = []; albums = []
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')

In [12]:
for tr in trs:
    rank = int(tr.select_one('.rank').string)
    info = tr.select_one('.ellipsis.rank01')
    title = info.find('a').string
    info = tr.select_one('.ellipsis.rank02')
    singer = info.find('a').string
    info = tr.select_one('.ellipsis.rank03')
    album = info.find('a').string
    
    entry = tr.select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100
    if len(spans) == 3:
        diff = int(spans[2].string)
        if diff != 0:
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1
    ranks.append(rank)
    diffs.append(diff)    
    titles.append(title)        
    singers.append(singer)
    albums.append(album)

In [13]:
Melon_top_100 = pd.DataFrame({
                    'rank' : ranks,
                    'diff' : diffs,
                    'title' : titles,
                    'singer' : singers,
                    'album' : albums
})
Melon_top_100 = Melon_top_100.set_index('rank')
print(f'기간 : {start_day} ~ {end_day}')
Melon_top_100

기간 : 2020.12.21 ~ 2020.12.27


,diff,title,singer,album
rank,,,,
1,0,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),미란이 (Mirani),쇼미더머니 9 Episode 1
2,4,All I Want For Christmas Is You,Mariah Carey,Merry Christmas (Deluxe Anniversary Edition)
3,0,밤하늘의 별을(2020),경서,밤하늘의 별을(2020)
4,-2,Dynamite,방탄소년단,Dynamite (DayTime Version)
5,7,Santa Tell Me,Ariana Grande,Santa Tell Me
...,...,...,...,...
96,-2,Maniac,Conan Gray,Maniac
97,-2,Paris In The Rain,Lauv,I met you when I was 18. (the playlist)
98,-100,그날에 나는 맘이 편했을까,이예준,그날에 나는 맘이 편했을까
